In [ ]:
! pip install -q kaggle

In [ ]:
# upload kaggle api token
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d 'trolukovich/apparel-images-dataset'

 98% 242M/248M [00:02<00:00, 106MB/s] 
100% 248M/248M [00:02<00:00, 90.4MB/s]


In [ ]:
%%bash
mkdir train
cd train
unzip '/content/apparel-images-dataset.zip'

In [ ]:
from tensorflow.keras.layers import *
from keras import backend as K
import tensorflow as tf
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
from keras import backend as K
import tensorflow as tf


def recall_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5),K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.clip(y_true, 0, 1))
    recall_ratio = true_positives / (possible_positives + K.epsilon())
    return recall_ratio

def precision_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5), K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(y_pred)
    precision_ratio = true_positives / (predicted_positives + K.epsilon())
    return precision_ratio

def f1_m(y_true, y_pred):
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.5,
                                   validation_split=0.05)

training_set = train_datagen.flow_from_directory('/content/training',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 subset='training')

val_set = train_datagen.flow_from_directory('/content/training',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 subset='validation')

Found 11385 images belonging to 24 classes.


In [ ]:
img_width, img_height = 299, 299

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
model = tf.keras.Sequential()
model.add( tf.keras.applications.Xception(
    input_shape=input_shape,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000))

model.add(Dense(24,activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',               
              metrics=['accuracy',f1_m])
model.summary()

91889664/91884032 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 1000)              22910480  
_________________________________________________________________
dense (Dense)                (None, 24)                24024     
Total params: 22,934,504
Trainable params: 22,879,976
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
c = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
history = model.fit_generator(generator=training_set,
                              validation_data = val_set,
                              epochs=100,
                              callbacks = [c],
                              verbose = 1)

In [ ]:
model.save('model.h5',)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()